In [15]:
import re
import pandas as pd 
from pprint import pprint

from nltk.corpus import stopwords
stopwords = stopwords.words('spanish')

import pyLDAvis.gensim_models
from gensim.models import LdaModel
from gensim.corpora import Dictionary

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [16]:
# Cargar datos
path = 'reviews_vidjew_es.csv'
data = pd.read_csv(path)
data.head()

,review_id,product_id,reviewer_id,stars,review_body,review_title,language,product_category
0,es_0825565,product_es_0370490,reviewer_es_0174781,3,"Buen. Buena calidad, y buena presentación.",Contenta,es,jewelry
1,es_0227934,product_es_0354224,reviewer_es_0411613,3,"Un producto a perfecto, para salir de casa con...",Versatilidad,es,video_games
2,es_0468601,product_es_0665460,reviewer_es_0348315,1,No funciona con Nintendo Switch. No hay forma ...,Decepción absoluta,es,video_games
3,es_0814494,product_es_0692692,reviewer_es_0951508,5,"Recomendado, los utilizo para pc y no me dan n...",Auriculares Pecham ps4,es,video_games
4,es_0206329,product_es_0728826,reviewer_es_0493255,4,El cable funciona bien podria ser un poco mas ...,Perfecto,es,video_games


In [17]:
def pre_procesado(texto):
    texto = texto.lower()
    texto = re.sub(r"[\W\d]+", " ", texto)
    texto = [palabra for palabra in texto.split() if palabra not in stopwords]
    return texto 

data['pp'] = data['review_body'].apply(lambda texto: pre_procesado(texto))

data.head()

,review_id,product_id,reviewer_id,stars,review_body,review_title,language,product_category,pp
0,es_0825565,product_es_0370490,reviewer_es_0174781,3,"Buen. Buena calidad, y buena presentación.",Contenta,es,jewelry,"[buen, buena, calidad, buena, presentación]"
1,es_0227934,product_es_0354224,reviewer_es_0411613,3,"Un producto a perfecto, para salir de casa con...",Versatilidad,es,video_games,"[producto, perfecto, salir, casa, nintendo, sw..."
2,es_0468601,product_es_0665460,reviewer_es_0348315,1,No funciona con Nintendo Switch. No hay forma ...,Decepción absoluta,es,video_games,"[funciona, nintendo, switch, forma, emparejarl..."
3,es_0814494,product_es_0692692,reviewer_es_0951508,5,"Recomendado, los utilizo para pc y no me dan n...",Auriculares Pecham ps4,es,video_games,"[recomendado, utilizo, pc, dan, ningún, proble..."
4,es_0206329,product_es_0728826,reviewer_es_0493255,4,El cable funciona bien podria ser un poco mas ...,Perfecto,es,video_games,"[cable, funciona, bien, podria, ser, mas, larg..."


In [18]:
# Crear una representación de los documentos en forma de diccionario
dictionary = Dictionary(data['pp'].values)
#dictionary.token2id  Le asigna a cada palarabra un numero
# Filtrar palabras muy frecuentes o infrecuentes
dictionary.filter_extremes(no_below=5, no_above=0.5)
corpus = [dictionary.doc2bow(text) for text in data['pp'].values]
#corpus
#Train the topic model
model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=7, passes=50, random_state = 28) #random es la semilla
#model

In [19]:
model.print_topics(num_words= 10)

[(0,
  '0.053*"bien" + 0.027*"si" + 0.023*"juego" + 0.021*"caja" + 0.017*"español" + 0.014*"cierre" + 0.014*"aunque" + 0.013*"buen" + 0.013*"viene" + 0.013*"va"'),
 (1,
  '0.057*"juego" + 0.030*"regalo" + 0.019*"bastante" + 0.019*"bien" + 0.019*"jugar" + 0.018*"bonito" + 0.016*"si" + 0.015*"gustado" + 0.014*"viene" + 0.014*"pena"'),
 (2,
  '0.042*"bonitos" + 0.034*"bien" + 0.033*"tamaño" + 0.026*"pendientes" + 0.026*"grandes" + 0.024*"quedan" + 0.021*"pequeños" + 0.016*"calidad" + 0.014*"demasiado" + 0.014*"esperaba"'),
 (3,
  '0.081*"calidad" + 0.056*"precio" + 0.039*"buena" + 0.024*"bien" + 0.022*"cadena" + 0.020*"foto" + 0.016*"aunque" + 0.015*"parece" + 0.014*"ve" + 0.014*"mala"'),
 (4,
  '0.038*"llegó" + 0.035*"perfecto" + 0.031*"producto" + 0.019*"juego" + 0.017*"buen" + 0.017*"precio" + 0.016*"bien" + 0.016*"si" + 0.015*"pedido" + 0.014*"juegos"'),
 (5,
  '0.033*"si" + 0.022*"así" + 0.021*"mando" + 0.019*"queda" + 0.019*"switch" + 0.017*"producto" + 0.017*"bien" + 0.015*"bonito"

In [22]:
lda_display = pyLDAvis.gensim_models.prepare(model, corpus, dictionary)
pyLDAvis.display(lda_display)

In [24]:
d = dictionary.doc2bow(["juego", "bien", "calidad"])
topics = [(cluster[0]+1, cluster[1]) for cluster in model.get_document_topics(d)]
topics


[(1, 0.035961706),
 (2, 0.3859157),
 (3, 0.035847858),
 (4, 0.43494788),
 (5, 0.035815116),
 (6, 0.03576708),
 (7, 0.03574467)]

In [31]:
def get_doc_top_n(text_processed, n):
    d = dictionary.doc2bow(text_processed)
    topics = dict(model.get_document_topics(d))
    try:
        return topics[n]
    except:
        return None

In [33]:
for t in range(0,7):
    top_name = f"topic_{t}"
    data[top_name] = data['pp'].apply(lambda doc: get_doc_top_n(doc, t))

for t in range(0,7):
    print(f"*********************************** TOPIC {t} ***********************************")
    topic = f"topic_{t}"
    for i,articulo in enumerate(data.sort_values('topic_0', ascending=False)['review_body'].values[:5]):
        print(f"review_body #{i}")
        print(articulo[:500])
        print()
    print()

*********************************** TOPIC 0 ***********************************
review_body #0
Quizá un poco caro por 25€ pero es que el juego es brutal. Uno de los mejores. Esta edición tiene todo el contenido disponible y funciona perfectamente en Xbox One y se instala en Español. La carátula es exactamente la que viene en la foto, la que pone Xbox One y Xbox 360. Pero OJO, es versión NTSC, si lo compras para tu Xbox 360 PAL no te funcionará

review_body #1
La consola llegó con la caja rota y desprecintada, y con signos de haber sido usada. En estas condiciones, no me ofrece ninguna confianza. Si el producto ha sido devuelto por algún motivo, deberían de avisarlo a la hora de la compra, y el cliente decidir si lo quiere en esas condiciones o no.

review_body #2
Engaño ,me han eviado una versión que no es uk o no viene en español,es una version polaca que solo viene en ingles polaco o ruso, dicen que al iniciar el juego se pone solo ,pero no , no hay manera de cambiarlo,y si tengo ps4